# Interpreting Psalms

## Filtering Psalms

In [ ]:
# Importing library
import json
import pandas as pd
import os

# Setting directory
# os.chdir('root_repository_path')
os.chdir('/Users/hiroshi/Documents/ds_projects/orare-model')

# Importing data
with open('bible/data/bible.json') as f:
    data = json.load(f)

In [ ]:
# Merging the verses in each chapter

# Filtering the "Salmos" book
psalms_data = [item for item in data if item['testamento'] == 'El Antiguo Testamento' and item['libro'] == 'SALMOS']

# Creating an empty list
psalms_chapters = []

# For loop to merge the verses in each chapter
for book in psalms_data:
   for chapter in book['capitulos']:
        merged_text = ''.join([verse['texto'] for verse in chapter['versiculos']])
        psalms_chapters.append({
            'capitulo': chapter['numero'],
            'texto': merged_text
        })

# Save psalms_chapters as JSON to a file
output_file = 'bible/data/psalms_chapters.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(psalms_chapters, f, ensure_ascii=False, indent=4)

In [ ]:
for i in psalms_chapters[:10]:
    print(f"Capitulo {i['capitulo']}: {i['texto']}")

## Interpreting Psalms with ChatGPT

In [ ]:
client = openai.OpenAI()

def get_completion_4o(prompt, model="gpt-4o"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [ ]:
# Prompt instruction
instructions = """
# Rol #
Tu eres el GPT: Catholic Bible Guide by Fr. Abraham Mutholath.
Tu rol será el de un sacerdote de una iglesia Católica. Se te dará distintos fragmentos de la Biblia. Tu tarea será leer los fragmentos y realizar las tareas que se detallen en la siguiente sección.
# Fin Rol #

# Tareas #
- Explicar la Biblia (explicacion): si el fragmento de la Biblia no está escrito en un lenguaje antiguo, es ambiguo o no está claro lo que quiere decir debes hacer una explicación breve y concisa. Solo explica lo que no es tan evidente y esté en un lenguaje antiguo o ambiguo.
- Identificar Area de la vida (areas): de cada fragmento debes identificar el área de la vida de la que se habla. Las áreas de la vida pueden ser: "Relaciones familiares/pareja","Amistad","Salud física","Salud mental/emocional","Desarrollo/crecimiento personal","Misión de vida","Dinero","Trabajo","Diversión/Hobbies","Otros". Para cada fragmento menciona entre 1 y 3 áreas que sean las más importantes.
# fin Tareas #

# Estructura input #
"id_libro | fragmento"
# Fin estructura input #

# Estructura Output # 
El output debe ser presentado en formato json con los siguiente campos:
- id_libro
- fragmento
- explicacion
- areas (en lista)
# Fin output #
"""

In [ ]:
# Creating sample to try the prompt
list_to_interpret = []
for i in range(5):
    text = f"salmos{psalms_chapters[i]['capitulo']} | {psalms_chapters[i]['texto']} "
    list_to_interpret.append(text)
print(list_to_interpret)

In [ ]:
# Merging the prompt with the sample data
prompt = f"""
{instructions}

# Input # 
{list_to_interpret}
# Fin Input #
"""

In [ ]:
# Interpreting the Bible with ChatGPT
response_4o = get_completion_4o(prompt)
print('gpt-4o')
print(' ')
print(response_4o)

In [ ]:
# Loading the response and saving as a plain text file
json_response_4o = json.loads(response_4o[7:-3])
df_response_4o = pd.DataFrame(json_response_4o)
#df_response_4o.to_csv('bible/data/psalms_sample_int_gpt4o.txt',sep='|',index=False, encoding='utf-8')

In [ ]:
# Printing the sample interpreted
for item in json_response_4o:
    print(f"Libro: {item['id_libro']}")
    print(f"Fragmento: {item['fragmento']}")
    print(f"Explicación: {item['explicacion']}")
    print(f"Áreas: {', '.join(item['areas'])}")
    print()